In [25]:
import os
import sys
import datetime
import glob as glob
import numpy as np
import cv2
import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import SGD
import tensorflow
#from scipy.interpolate import spline
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# DATA LOADING

In [26]:
# default settings
img_width, img_height = 224, 224

train_dir = '/home/dell/Downloads/train/'
validate_dir = '/home/dell/Downloads/val/'
nb_epochs = 20
batch_size = 2
nb_classes = len(glob.glob(train_dir + '/*'))

In [27]:
# get number of images in training directory
nb_train_samples = 0
for r, dirs, files in os.walk(train_dir):
    for dr in dirs:
        nb_train_samples += len(glob.glob(os.path.join(r, dr + "/*")))
print(nb_train_samples)

297


In [28]:
# get number of images in validation directory
nb_validate_samples = 0
for r, dirs, files in os.walk(validate_dir):
    for dr in dirs:
        nb_validate_samples += len(glob.glob(os.path.join(r, dr + "/*")))
print(nb_validate_samples)

33


# DATA AUGMENTATION

In [29]:
# data pre-processing for training
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [30]:
# data pre-processing for validation
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [31]:
# generate and store training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 297 images belonging to 2 classes.


In [32]:
# generate and store validation data
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)



Found 33 images belonging to 2 classes.


# LOADING THE MODEL

In [33]:


# set up transfer learning on pre-trained ImageNet VGG16 model 
vgg16_model = VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'sigmoid')(x)
model = Model(input = vgg16_model.input, output = predictions)



/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [34]:
# freeze all layers of the pre-trained model
for layer in vgg16_model.layers:
    layer.trainable = False

In [35]:
# compile the new model using a RMSProp optimizer
model.compile(optimizer = 'rmsprop',
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

# TRAINING

In [36]:
# fit the model, log the results and the training time
now = datetime.datetime.now
t = now()
transfer_learning_history = model.fit_generator(
    train_generator,
    nb_epoch = nb_epochs,
    samples_per_epoch = nb_train_samples,
    validation_data = validate_generator,
    nb_val_samples = nb_validate_samples,
    class_weight='auto')
print('Training time: %s' % (now() - t))

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  # Remove the CWD from sys.path while we load stuff.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., class_weight="auto", steps_per_epoch=148, epochs=20, validation_steps=33)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/20
148/148 [==============================] - 107s 722ms/step - loss: 0.3039 - accuracy: 0.8831 - val_loss: 0.0488 - val_accuracy: 0.9692
Epoch 2/20
148/148 [==============================] - 109s 735ms/step - loss: 0.1191 - accuracy: 0.9559 - val_loss: 0.0013 - val_accuracy: 0.9688
Epoch 3/20
148/148 [==============================] - 112s 755ms/step - loss: 0.0979 - accuracy: 0.9729 - val_loss: 0.0230 - val_accuracy: 1.0000
Epoch 4/20
148/148 [==============================] - 111s 747ms/step - loss: 0.0814 - accuracy: 0.9729 - val_loss: 0.0074 - val_accuracy: 0.9688
Epoch 5/20
148/148 [==============================] - 111s 747ms/step - loss: 0.0716 - accuracy: 0.9763 - val_loss: 3.2738e-05 - val_accuracy: 0.9844
Epoch 6/20
148/148 [==============================] - 112s 758ms/step - loss: 0.0826 - accuracy: 0.9695 - val_loss: 2.1919e-04 - val_accuracy: 1.0000
Epoch 7/20
148/148 [==============================] - 107s 725ms/step - loss: 0.0554 - accuracy: 0.9797 - val_loss: 

In [37]:
# evaluate the performance the new model and report the results
score = model.evaluate_generator(validate_generator, nb_validate_samples)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

Test Loss: 0.6562985181808472
Test Accuracy: 0.9692307710647583


# FINE TUNING

In [11]:
# Step 1 - Set up fine tuning on pre-trained ImageNet VGG16 model - train all layers
for layer in model.layers:
    layer.trainable = True
    
# Step 2 - Compile the revised model using SGD optimizer with a learing rate of 0.0001 and a momentum of 0.9
model.compile(optimizer = SGD(lr = 0.0001, momentum=0.9), 
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

# Step 3 - Fit the revised model, log the results and the training time
now = datetime.datetime.now
t = now()
fine_tuning_history = model.fit_generator(
    train_generator,
    nb_epoch = nb_epochs,
    samples_per_epoch = nb_train_samples,
    validation_data = validate_generator,
    nb_val_samples = nb_validate_samples,
    class_weight='auto')
print('Training time: %s' % (now() - t))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., class_weight="auto", steps_per_epoch=164, epochs=20, validation_steps=27)`



Epoch 1/20
164/164 [==============================] - 405s 2s/step - loss: 0.4932 - accuracy: 0.7768 - val_loss: 0.2438 - val_accuracy: 0.9811
Epoch 2/20
164/164 [==============================] - 406s 2s/step - loss: 0.1931 - accuracy: 0.9343 - val_loss: 0.0271 - val_accuracy: 0.9808
Epoch 3/20
164/164 [==============================] - 401s 2s/step - loss: 0.1128 - accuracy: 0.9557 - val_loss: 0.0122 - val_accuracy: 1.0000
Epoch 4/20
164/164 [==============================] - 397s 2s/step - loss: 0.1045 - accuracy: 0.9602 - val_loss: 0.0215 - val_accuracy: 0.9808
Epoch 5/20
164/164 [==============================] - 407s 2s/step - loss: 0.0775 - accuracy: 0.9725 - val_loss: 0.1560 - val_accuracy: 0.9904
Epoch 6/20
164/164 [==============================] - 401s 2s/step - loss: 0.0692 - accuracy: 0.9847 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 7/20
164/164 [==============================] - 403s 2s/step - loss: 0.0770 - accuracy: 0.9817 - val_loss: 0.0026 - val_accuracy: 1.000

In [12]:
# evaluate the performance the new model and report the results
score = model.evaluate_generator(validate_generator, nb_validate_samples)
print("Test Loss:", score[0])
print("Test Accuracy:", score[1])

Test Loss: 7.132034807000309e-05
Test Accuracy: 1.0
